In [ ]:
import json
from datascience import *
import os
import datetime

In [ ]:
fp = '/Users/leviharris/Library/CloudStorage/GoogleDrive-leviharris555@gmail.com/Other computers/mac_new/NBA_HUDL_data/nba-plus-statvu-dataset/statvu-game-logs/01.20.2016.CHA.at.OKC/0021500636.json'
with open(fp, 'r') as f:
    data = json.load(f)

In [ ]:
data['events'][-1]['moments'][-1]

In [210]:
import tqdm
import pandas as pd
import numpy as np

statvu_logs_dir = r'/Users/leviharris/Library/CloudStorage/GoogleDrive-leviharris555@gmail.com/Other computers/mac_new/NBA_HUDL_data/nba-plus-statvu-dataset/statvu-game-logs'
statvu_logs = os.listdir(statvu_logs_dir)
statvu_logs_paths = [os.path.join(statvu_logs_dir, f) for f in statvu_logs]
example = statvu_logs[0].split('.')

statvu_dates = []
statvu_teams = []
statvu_fps = []
statvu_game_names = []
statvu_games = set()

hudl_dates = []
hudl_teams = []
hudl_ids = []
hudl_fps = []
hudl_names = []

for log in statvu_logs:
    s = log.split('.')
    fp = os.path.join(statvu_logs_dir, log)
    try:
        date = datetime.date(int(s[2]), int(s[0]), int(s[1]))
        team_1 = s[3]

        # some clever bs
        try:
            int(s[5])
            team_2 = s[4]
        except:
            team_2 = s[5]

        teams_arr = sorted([team_1, team_2])
        _teams = ', '.join(teams_arr)
        game = str(date) + ' ' + _teams
        if game not in statvu_games:
            statvu_games.add(game)
            statvu_dates.append(date)
            statvu_teams.append(_teams)
            statvu_fps.append(fp)
            statvu_game_names.append(log.replace('.7z', ''))
    except:
        pass

statvu_logs: Table = Table().with_columns(
    'statvu_date', statvu_dates,
    'statvu_teams', statvu_teams,
    'statvu_fps', statvu_fps
)

In [211]:
hudl_logs_dir = r'/Users/leviharris/Library/CloudStorage/GoogleDrive-leviharris555@gmail.com/Other computers/mac_new/NBA_HUDL_data/nba-plus-statvu-dataset/hudl-game-logs'
hudl_logs = os.listdir(hudl_logs_dir)

for log in hudl_logs:
    if not '.csv' in log or not '-' in log:
        continue
    try:
        s = log.split('.')
        date_s = s[0].split('-')
        date = datetime.date(
            int(date_s[2]),
            int(date_s[0]),
            int(date_s[1]),
        )
        t1 = s[1]
        t2 = s[2]
        _id = s[-2]
        hudl_dates.append(date)
        hudl_teams.append(', '.join(sorted([t1, t2])))
        hudl_ids.append(_id)
        hudl_fps.append(
            os.path.join(hudl_logs_dir, log)
        )
        hudl_names.append(log.replace('.csv', ''))
    except:
        print(log)
        break

hudl_logs = Table().with_columns(
    'hudl_date', hudl_dates,
    'hudl_ids', hudl_ids,
    'hudl_teams', hudl_teams,
    'hudl_fps', hudl_fps,
    'hudl_name', hudl_names,
)

closest_matches = []
closest_hudl_fps = []
closest_hudl_names = []

for row in tqdm.tqdm(statvu_logs.rows):

    closest_path = ''
    closest_hudl_name = ''
    closet_date = 1000
    clostest_match = -1
    date_one = row[0]
    team_one = row[1]

    for _row in hudl_logs.rows:
        date_two = _row[0]
        diff = str(abs((date_one - date_two)))
        if 'day' in diff:
            diff = int(diff.split(' day')[0])
        else:
            diff = 0
        team_two = _row[2]
        if team_one == team_two and diff <= 1:
            closet_date = min(closet_date, diff)
            clostest_match = _row[1]
            closest_path = _row[3]
            closest_hudl_name = _row[4]

    closest_matches.append(clostest_match)
    closest_hudl_fps.append(closest_path)
    closest_hudl_names.append(closest_hudl_name)

100%|██████████| 634/634 [00:01<00:00, 370.52it/s]


In [223]:
_dates = statvu_logs.column(0)
_teams = statvu_logs.column(1)

statvu_logs_with_matches = Table().with_columns(
    'game_id', closest_matches,
    'statvu_game_name', statvu_game_names,
    'hudl_game_name', closest_hudl_names,
    'statvu_date', _dates,
    'statvu_teams', _teams,
    'statvu_fp', statvu_fps,
    'hudl_fp', closest_hudl_fps
)
statvu_logs_with_matches.sort('game_id')

game_id,statvu_game_name,hudl_game_name,statvu_date,statvu_teams,statvu_fp,hudl_fp
-1,12.31.2015.LAC.at.NOP,,2015-12-31,"LAC, NOP",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,11.30.2015.POR.at.LAC,,2015-11-30,"LAC, POR",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,01.22.2016.LAC.at.NYK,,2016-01-22,"LAC, NYK",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,01.23.2016.UTA.at.WAS,,2016-01-23,"UTA, WAS",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,12.07.2015.LAC.at.MIN,,2015-12-07,"LAC, MIN",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,01.10.2016.NOP.at.LAC,,2016-01-10,"LAC, NOP",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,12.21.2015.OKC.at.LAC,,2015-12-21,"LAC, OKC",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,10.31.2015.SAC.at.LAC,,2015-10-31,"LAC, SAC",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,11.04.2015.LAC.at.GSW,,2015-11-04,"GSW, LAC",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,
-1,10.28.2015.LAC.at.SAC,,2015-10-28,"LAC, SAC",/Users/leviharris/Library/CloudStorage/GoogleDrive-levih ...,


In [238]:
files = os.listdir(statvu_logs_dir)

i = 0
new_names = set()
for file in files:
    if file.endswith('.7z') or '.' not in file or 'DS' in file:
        continue
    fp = os.path.join(statvu_logs_dir, file)
    t: Table = statvu_logs_with_matches.where('statvu_game_name', are.equal_to(file))
    if t.num_rows == 1 and t.column('game_id')[0] != '-1':
        new_name = t.column('hudl_game_name')[0]
        if new_name not in new_names:
            try:
                new_names.add(new_name)
                new_path = os.path.join(statvu_logs_dir, new_name)
                os.rename(fp, new_path)
                # print(f"Old path: {fp}")
                # print(f"New path: {new_path}")
                # print('___________________________________')
            except:
                print(f'Error: {new_path} already exists!')


i, len(new_names)


Error: /Users/leviharris/Library/CloudStorage/GoogleDrive-leviharris555@gmail.com/Other computers/mac_new/NBA_HUDL_data/nba-plus-statvu-dataset/statvu-game-logs/10-31-2015.POR.PHX.18132 already exists!
Error: /Users/leviharris/Library/CloudStorage/GoogleDrive-leviharris555@gmail.com/Other computers/mac_new/NBA_HUDL_data/nba-plus-statvu-dataset/statvu-game-logs/11-21-2015.BOS.BKN.21574 already exists!


(0, 434)